<p>Euler's totient function, $\phi(n)$ [sometimes called the phi function], is defined as the number of positive integers not exceeding $n$ which are relatively prime to $n$. For example, as $1$, $2$, $4$, $5$, $7$, and $8$, are all less than or equal to nine and relatively prime to nine, $\phi(9)=6$.</p>
<div class="center">
<table class="grid center"><tr><td><b>$n$</b></td>
<td><b>Relatively Prime</b></td>
<td><b>$\phi(n)$</b></td>
<td><b>$n/\phi(n)$</b></td>
</tr><tr><td>2</td>
<td>1</td>
<td>1</td>
<td>2</td>
</tr><tr><td>3</td>
<td>1,2</td>
<td>2</td>
<td>1.5</td>
</tr><tr><td>4</td>
<td>1,3</td>
<td>2</td>
<td>2</td>
</tr><tr><td>5</td>
<td>1,2,3,4</td>
<td>4</td>
<td>1.25</td>
</tr><tr><td>6</td>
<td>1,5</td>
<td>2</td>
<td>3</td>
</tr><tr><td>7</td>
<td>1,2,3,4,5,6</td>
<td>6</td>
<td>1.1666...</td>
</tr><tr><td>8</td>
<td>1,3,5,7</td>
<td>4</td>
<td>2</td>
</tr><tr><td>9</td>
<td>1,2,4,5,7,8</td>
<td>6</td>
<td>1.5</td>
</tr><tr><td>10</td>
<td>1,3,7,9</td>
<td>4</td>
<td>2.5</td>
</tr></table></div>
<p>It can be seen that $n = 6$ produces a maximum $n/\phi(n)$ for $n\leq 10$.</p>
<p>Find the value of $n\leq 1\,000\,000$ for which $n/\phi(n)$ is a maximum.</p>

In [ ]:
'''
Resources:
- https://en.wikipedia.org/wiki/Coprime_integers
- https://en.wikipedia.org/wiki/Euler%27s_totient_function
- https://en.wikipedia.org/wiki/Euclidean_algorithm
'''

# GCD Algorithm
#def gcd(a: int, b: int) -> int:
#    return a if b == 0 else gcd(b, a % b)

gcd: int = lambda a, b: a if b == 0 else gcd(b, a % b)

if __name__ == "__main__":
    print("{}".format(gcd(2, 4)))

In [72]:
# MAIN PROGRAM

'''
TODO:
- will give correct result, but algorithm is too slow for large N

- using a 2D lookup table: gcd(a, b) -> {x:{...}, ..., a:{x: c, b: n}, ...}
does not speed up the process either, as shown below. There are too many redundant
entries for consecutive and adjacent n values. A more mathematical and ultimately simpler
approach will be used next to solve the problem.
'''

MAX_N: int = 2000

#gcd: int = lambda a, b: a if b == 0 else gcd(b, a % b)
gcd_cache: dict = {}

def gcd(a: int, b: int) -> int:
    '''
    - # gcd: int = lambda a, b: a if b == 0 else gcd(b, a % b)
    - # (recursive) Euclidean algorithm to compute GCD
    
    # Ex. n = 8, thus coprimes of 8 and [1,8] are: 1, 3, 5, 7

    # table[a][b] -> gcd(a, b)
    # table can list all coprimes OR show the count of coprimes
    # note that gcd(a, b) = gcd(b, a)

    # dic: dict = {"2": {"1": 1, "2": 2, "3": 1, "14": 2}, "3": {"1": 1, "2": 1, "23": 1}, ...}

    - "2": {"3": x} == "3": {"2": x} so when checking table, let [i, j] be [min(a, b), max(a, b)]
    '''
    # check lookup table
    min_val: int = min(a, b)
    max_val: int = max(a, b)
    if min_val in gcd_cache and max_val in gcd_cache[min_val]:
        return gcd_cache[min_val][max_val]
    
    # compute gcd
    ret_val: int = a if b == 0 else gcd(b, a % b)
    
    # cache the new value and return
    if min_val not in gcd_cache:
        gcd_cache.update({min_val: {}})
    if max_val not in gcd_cache[min_val]:
        gcd_cache[min_val].update({min_val: {max_val: ret_val}})
    return ret_val

if __name__ == "__main__":
    max_n: int = 0
    ret_frac: float = 0.0
    for i in range(2, MAX_N):
        coprime_count: int = 0
        for j in range(1, i + 1):
            if gcd(i, j) == 1:
                coprime_count += 1
        print("n = {0}, phi(n) = {1}, n / phi(n) = {2}".format(i, coprime_count, float(i / coprime_count)))

        #ret_frac = max(ret_frac, float(i / coprime_count))
        if float(i / coprime_count) > ret_frac:
            ret_frac = float(i / coprime_count)
            max_n = i
    print("\n[n, max(n / phi(n))]: [{0}, {1}]".format(max_n, ret_frac))

n = 2, phi(n) = 1, n / phi(n) = 2.0
n = 3, phi(n) = 2, n / phi(n) = 1.5
n = 4, phi(n) = 2, n / phi(n) = 2.0
n = 5, phi(n) = 4, n / phi(n) = 1.25
n = 6, phi(n) = 2, n / phi(n) = 3.0
n = 7, phi(n) = 6, n / phi(n) = 1.1666666666666667
n = 8, phi(n) = 4, n / phi(n) = 2.0
n = 9, phi(n) = 6, n / phi(n) = 1.5
n = 10, phi(n) = 4, n / phi(n) = 2.5
n = 11, phi(n) = 10, n / phi(n) = 1.1
n = 12, phi(n) = 4, n / phi(n) = 3.0
n = 13, phi(n) = 12, n / phi(n) = 1.0833333333333333
n = 14, phi(n) = 6, n / phi(n) = 2.3333333333333335
n = 15, phi(n) = 8, n / phi(n) = 1.875
n = 16, phi(n) = 8, n / phi(n) = 2.0
n = 17, phi(n) = 16, n / phi(n) = 1.0625
n = 18, phi(n) = 6, n / phi(n) = 3.0
n = 19, phi(n) = 18, n / phi(n) = 1.0555555555555556
n = 20, phi(n) = 8, n / phi(n) = 2.5
n = 21, phi(n) = 12, n / phi(n) = 1.75
n = 22, phi(n) = 10, n / phi(n) = 2.2
n = 23, phi(n) = 22, n / phi(n) = 1.0454545454545454
n = 24, phi(n) = 8, n / phi(n) = 3.0
n = 25, phi(n) = 20, n / phi(n) = 1.25
n = 26, phi(n) = 12, n / phi(

n = 287, phi(n) = 240, n / phi(n) = 1.1958333333333333
n = 288, phi(n) = 96, n / phi(n) = 3.0
n = 289, phi(n) = 272, n / phi(n) = 1.0625
n = 290, phi(n) = 112, n / phi(n) = 2.5892857142857144
n = 291, phi(n) = 192, n / phi(n) = 1.515625
n = 292, phi(n) = 144, n / phi(n) = 2.0277777777777777
n = 293, phi(n) = 292, n / phi(n) = 1.0034246575342465
n = 294, phi(n) = 84, n / phi(n) = 3.5
n = 295, phi(n) = 232, n / phi(n) = 1.271551724137931
n = 296, phi(n) = 144, n / phi(n) = 2.0555555555555554
n = 297, phi(n) = 180, n / phi(n) = 1.65
n = 298, phi(n) = 148, n / phi(n) = 2.0135135135135136
n = 299, phi(n) = 264, n / phi(n) = 1.1325757575757576
n = 300, phi(n) = 80, n / phi(n) = 3.75
n = 301, phi(n) = 252, n / phi(n) = 1.1944444444444444
n = 302, phi(n) = 150, n / phi(n) = 2.013333333333333
n = 303, phi(n) = 200, n / phi(n) = 1.515
n = 304, phi(n) = 144, n / phi(n) = 2.111111111111111
n = 305, phi(n) = 240, n / phi(n) = 1.2708333333333333
n = 306, phi(n) = 96, n / phi(n) = 3.1875
n = 307, phi

n = 481, phi(n) = 432, n / phi(n) = 1.1134259259259258
n = 482, phi(n) = 240, n / phi(n) = 2.0083333333333333
n = 483, phi(n) = 264, n / phi(n) = 1.8295454545454546
n = 484, phi(n) = 220, n / phi(n) = 2.2
n = 485, phi(n) = 384, n / phi(n) = 1.2630208333333333
n = 486, phi(n) = 162, n / phi(n) = 3.0
n = 487, phi(n) = 486, n / phi(n) = 1.0020576131687242
n = 488, phi(n) = 240, n / phi(n) = 2.033333333333333
n = 489, phi(n) = 324, n / phi(n) = 1.5092592592592593
n = 490, phi(n) = 168, n / phi(n) = 2.9166666666666665
n = 491, phi(n) = 490, n / phi(n) = 1.0020408163265306
n = 492, phi(n) = 160, n / phi(n) = 3.075
n = 493, phi(n) = 448, n / phi(n) = 1.1004464285714286
n = 494, phi(n) = 216, n / phi(n) = 2.287037037037037
n = 495, phi(n) = 240, n / phi(n) = 2.0625
n = 496, phi(n) = 240, n / phi(n) = 2.066666666666667
n = 497, phi(n) = 420, n / phi(n) = 1.1833333333333333
n = 498, phi(n) = 164, n / phi(n) = 3.0365853658536586
n = 499, phi(n) = 498, n / phi(n) = 1.002008032128514
n = 500, phi(n

n = 658, phi(n) = 276, n / phi(n) = 2.3840579710144927
n = 659, phi(n) = 658, n / phi(n) = 1.0015197568389058
n = 660, phi(n) = 160, n / phi(n) = 4.125
n = 661, phi(n) = 660, n / phi(n) = 1.0015151515151515
n = 662, phi(n) = 330, n / phi(n) = 2.006060606060606
n = 663, phi(n) = 384, n / phi(n) = 1.7265625
n = 664, phi(n) = 328, n / phi(n) = 2.024390243902439
n = 665, phi(n) = 432, n / phi(n) = 1.5393518518518519
n = 666, phi(n) = 216, n / phi(n) = 3.0833333333333335
n = 667, phi(n) = 616, n / phi(n) = 1.0827922077922079
n = 668, phi(n) = 332, n / phi(n) = 2.0120481927710845
n = 669, phi(n) = 444, n / phi(n) = 1.5067567567567568
n = 670, phi(n) = 264, n / phi(n) = 2.537878787878788
n = 671, phi(n) = 600, n / phi(n) = 1.1183333333333334
n = 672, phi(n) = 192, n / phi(n) = 3.5
n = 673, phi(n) = 672, n / phi(n) = 1.0014880952380953
n = 674, phi(n) = 336, n / phi(n) = 2.005952380952381
n = 675, phi(n) = 360, n / phi(n) = 1.875
n = 676, phi(n) = 312, n / phi(n) = 2.1666666666666665
n = 677, 

n = 836, phi(n) = 360, n / phi(n) = 2.3222222222222224
n = 837, phi(n) = 540, n / phi(n) = 1.55
n = 838, phi(n) = 418, n / phi(n) = 2.0047846889952154
n = 839, phi(n) = 838, n / phi(n) = 1.0011933174224343
n = 840, phi(n) = 192, n / phi(n) = 4.375
n = 841, phi(n) = 812, n / phi(n) = 1.0357142857142858
n = 842, phi(n) = 420, n / phi(n) = 2.0047619047619047
n = 843, phi(n) = 560, n / phi(n) = 1.5053571428571428
n = 844, phi(n) = 420, n / phi(n) = 2.0095238095238095
n = 845, phi(n) = 624, n / phi(n) = 1.3541666666666667
n = 846, phi(n) = 276, n / phi(n) = 3.0652173913043477
n = 847, phi(n) = 660, n / phi(n) = 1.2833333333333334
n = 848, phi(n) = 416, n / phi(n) = 2.0384615384615383
n = 849, phi(n) = 564, n / phi(n) = 1.5053191489361701
n = 850, phi(n) = 320, n / phi(n) = 2.65625
n = 851, phi(n) = 792, n / phi(n) = 1.0744949494949494
n = 852, phi(n) = 280, n / phi(n) = 3.0428571428571427
n = 853, phi(n) = 852, n / phi(n) = 1.0011737089201878
n = 854, phi(n) = 360, n / phi(n) = 2.3722222222

...

n = 1194, phi(n) = 396, n / phi(n) = 3.015151515151515
n = 1195, phi(n) = 952, n / phi(n) = 1.2552521008403361
n = 1196, phi(n) = 528, n / phi(n) = 2.265151515151515
n = 1197, phi(n) = 648, n / phi(n) = 1.8472222222222223
n = 1198, phi(n) = 598, n / phi(n) = 2.0033444816053514
n = 1199, phi(n) = 1080, n / phi(n) = 1.1101851851851852
n = 1200, phi(n) = 320, n / phi(n) = 3.75
n = 1201, phi(n) = 1200, n / phi(n) = 1.0008333333333332
n = 1202, phi(n) = 600, n / phi(n) = 2.0033333333333334
n = 1203, phi(n) = 800, n / phi(n) = 1.50375
n = 1204, phi(n) = 504, n / phi(n) = 2.388888888888889
n = 1205, phi(n) = 960, n / phi(n) = 1.2552083333333333
n = 1206, phi(n) = 396, n / phi(n) = 3.0454545454545454
n = 1207, phi(n) = 1120, n / phi(n) = 1.0776785714285715
n = 1208, phi(n) = 600, n / phi(n) = 2.013333333333333
n = 1209, phi(n) = 720, n / phi(n) = 1.6791666666666667
n = 1210, phi(n) = 440, n / phi(n) = 2.75
n = 1211, phi(n) = 1032, n / phi(n) = 1.1734496124031009
n = 1212, phi(n) = 400, n / phi

n = 1368, phi(n) = 432, n / phi(n) = 3.1666666666666665
n = 1369, phi(n) = 1332, n / phi(n) = 1.0277777777777777
n = 1370, phi(n) = 544, n / phi(n) = 2.5183823529411766
n = 1371, phi(n) = 912, n / phi(n) = 1.5032894736842106
n = 1372, phi(n) = 588, n / phi(n) = 2.3333333333333335
n = 1373, phi(n) = 1372, n / phi(n) = 1.000728862973761
n = 1374, phi(n) = 456, n / phi(n) = 3.013157894736842
n = 1375, phi(n) = 1000, n / phi(n) = 1.375
n = 1376, phi(n) = 672, n / phi(n) = 2.0476190476190474
n = 1377, phi(n) = 864, n / phi(n) = 1.59375
n = 1378, phi(n) = 624, n / phi(n) = 2.2083333333333335
n = 1379, phi(n) = 1176, n / phi(n) = 1.1726190476190477
n = 1380, phi(n) = 352, n / phi(n) = 3.9204545454545454
n = 1381, phi(n) = 1380, n / phi(n) = 1.0007246376811594
n = 1382, phi(n) = 690, n / phi(n) = 2.0028985507246375
n = 1383, phi(n) = 920, n / phi(n) = 1.5032608695652174
n = 1384, phi(n) = 688, n / phi(n) = 2.011627906976744
n = 1385, phi(n) = 1104, n / phi(n) = 1.2545289855072463
n = 1386, phi

n = 1532, phi(n) = 764, n / phi(n) = 2.005235602094241
n = 1533, phi(n) = 864, n / phi(n) = 1.7743055555555556
n = 1534, phi(n) = 696, n / phi(n) = 2.204022988505747
n = 1535, phi(n) = 1224, n / phi(n) = 1.2540849673202614
n = 1536, phi(n) = 512, n / phi(n) = 3.0
n = 1537, phi(n) = 1456, n / phi(n) = 1.0556318681318682
n = 1538, phi(n) = 768, n / phi(n) = 2.0026041666666665
n = 1539, phi(n) = 972, n / phi(n) = 1.5833333333333333
n = 1540, phi(n) = 480, n / phi(n) = 3.2083333333333335
n = 1541, phi(n) = 1452, n / phi(n) = 1.0612947658402203
n = 1542, phi(n) = 512, n / phi(n) = 3.01171875
n = 1543, phi(n) = 1542, n / phi(n) = 1.0006485084306096
n = 1544, phi(n) = 768, n / phi(n) = 2.0104166666666665
n = 1545, phi(n) = 816, n / phi(n) = 1.8933823529411764
n = 1546, phi(n) = 772, n / phi(n) = 2.0025906735751295
n = 1547, phi(n) = 1152, n / phi(n) = 1.3428819444444444
n = 1548, phi(n) = 504, n / phi(n) = 3.0714285714285716
n = 1549, phi(n) = 1548, n / phi(n) = 1.0006459948320414
n = 1550, p

n = 1689, phi(n) = 1124, n / phi(n) = 1.5026690391459074
n = 1690, phi(n) = 624, n / phi(n) = 2.7083333333333335
n = 1691, phi(n) = 1584, n / phi(n) = 1.067550505050505
n = 1692, phi(n) = 552, n / phi(n) = 3.0652173913043477
n = 1693, phi(n) = 1692, n / phi(n) = 1.0005910165484633
n = 1694, phi(n) = 660, n / phi(n) = 2.566666666666667
n = 1695, phi(n) = 896, n / phi(n) = 1.8917410714285714
n = 1696, phi(n) = 832, n / phi(n) = 2.0384615384615383
n = 1697, phi(n) = 1696, n / phi(n) = 1.0005896226415094
n = 1698, phi(n) = 564, n / phi(n) = 3.0106382978723403
n = 1699, phi(n) = 1698, n / phi(n) = 1.0005889281507656
n = 1700, phi(n) = 640, n / phi(n) = 2.65625
n = 1701, phi(n) = 972, n / phi(n) = 1.75
n = 1702, phi(n) = 792, n / phi(n) = 2.148989898989899
n = 1703, phi(n) = 1560, n / phi(n) = 1.0916666666666666
n = 1704, phi(n) = 560, n / phi(n) = 3.0428571428571427
n = 1705, phi(n) = 1200, n / phi(n) = 1.4208333333333334
n = 1706, phi(n) = 852, n / phi(n) = 2.0023474178403755
n = 1707, phi

n = 1842, phi(n) = 612, n / phi(n) = 3.0098039215686274
n = 1843, phi(n) = 1728, n / phi(n) = 1.0665509259259258
n = 1844, phi(n) = 920, n / phi(n) = 2.0043478260869567
n = 1845, phi(n) = 960, n / phi(n) = 1.921875
n = 1846, phi(n) = 840, n / phi(n) = 2.197619047619048
n = 1847, phi(n) = 1846, n / phi(n) = 1.0005417118093174
n = 1848, phi(n) = 480, n / phi(n) = 3.85
n = 1849, phi(n) = 1806, n / phi(n) = 1.0238095238095237
n = 1850, phi(n) = 720, n / phi(n) = 2.5694444444444446
n = 1851, phi(n) = 1232, n / phi(n) = 1.5024350649350648
n = 1852, phi(n) = 924, n / phi(n) = 2.0043290043290045
n = 1853, phi(n) = 1728, n / phi(n) = 1.072337962962963
n = 1854, phi(n) = 612, n / phi(n) = 3.0294117647058822
n = 1855, phi(n) = 1248, n / phi(n) = 1.486378205128205
n = 1856, phi(n) = 896, n / phi(n) = 2.0714285714285716
n = 1857, phi(n) = 1236, n / phi(n) = 1.5024271844660195
n = 1858, phi(n) = 928, n / phi(n) = 2.002155172413793
n = 1859, phi(n) = 1560, n / phi(n) = 1.1916666666666667
n = 1860, ph

In [37]:
gcd_cache

{0: {1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99},
 1: {2: 1,
  3: 1,
  4: 

In [77]:
# NEW APPROACH

'''
- note, this approach requires an access to prime numbers.
The calculation of n/phi(n) produces a higher ratio when phi(n) is minimal,
indicating that prime numbers are likely to have more coprimes thus reducing
the ratio.

The Totient Function is based on the distribution of distinct primes that divide n
'''

def prime_factors(num: int) -> list:
    '''
    - generates the list of prime factors of a given 
    positive integer num
    '''
    prime_factors: list = []
    for term in range(2, num):
        while num % term == 0:
            if num == 1:
                return
            num = num / term
            prime_factors.append(term)
    return prime_factors

def generate_primes(num: int) -> list:
    '''
    - generates a list of the first num prime numbers
    '''
    primes: list = []
    for n in range(2, num + 1):
        is_prime: bool = True
        for i in range(2, int(n ** 0.5) + 1):
            if n % i == 0:
                is_prime = False
                break
        if is_prime == True:
            primes.append(n)
    return primes

# Example usage:
if __name__ == "__main__":
    n = 11
    prime_numbers = generate_primes(n)
    print(prime_numbers)

    print(prime_factors(900))

[2, 3, 5, 7, 11]
[2, 2, 3, 3, 5, 5]


In [83]:
# MAIN PROGRAM

# list of distinct primes whose product does not exceed 1000000
LIMIT: int = 1000000
PRIMES: list = generate_primes(100)

if __name__ == "__main__":
    '''
    - including more distinct primes in n will increase
    the desired ratio.
    '''
    n: int = 1
    k: int = 0
    while PRIMES[k] * n <= LIMIT:
        n *= PRIMES[k]
        k += 1
    print("n where max(n / phi(n)): {}".format(n))

n where max(n / phi(n)): 510510
